In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    argv = list(argv)
    argv.insert(0, '--num-workers=4')
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
torch.cuda.empty_cache()

Hyperparams to test:

1. bounding box size
2. ~batch size~
3. ~learning rate~
4. ~steps per epoch~
5. windowing
6. augmentation
7. model depth
8. weight decay
9. pad type

In [5]:
experiment_epochs = 3
val_cadence = 1
batch_size = 32
recall_priority = 0
pad_types = ['zero', 'reflect', 'replicate']
windows = [None, 'lung', 'mediastinal', 'dist']

# BBox (7,60,60)

We will first try to run tests using a (7,60,60) bounding box

In [4]:
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '60', '60')

2020-11-13 18:48:24,751 INFO     pid:49136 nb:004:run Running: prepcache.CovidPrepCacheApp(['--num-workers=4', '--width-irc', '7', '60', '60']).main()
2020-11-13 18:48:25,679 INFO     pid:49136 prepcache:040:main Starting CovidPrepCacheApp, Namespace(batch_size=1024, num_workers=4, width_irc=['7', '60', '60'])
100%|██████████| 2/2 [01:03<00:00, 31.57s/it]
2020-11-13 18:49:28,889 INFO     pid:49136 nb:009:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=4', '--width-irc', '7', '60', '60'].main()


## First Run (default hyperparams)

In [6]:
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--run-name=bbox60')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-11-14 14:04:31,053 INFO     pid:15525 nb:004:run Running: training.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=3', '--batch-size=32', '--run-name=bbox60']).main()
2020-11-14 14:04:32,457 INFO     pid:15525 training:184:init_model Using CUDA; 1 devices.
2020-11-14 14:04:35,100 INFO     pid:15525 modules.dsets:185:__init__ TrainingCovid2dSegmentationDataset: training mode, 177 uid's, 4379 index slices, 1080 lesions
2020-11-14 14:04:35,101 INFO     pid:15525 modules.dsets:226:__init__ TrainingCovid2dSegmentationDataset: (7, 60, 60) width_irc
2020-11-14 14:04:35,108 INFO     pid:15525 modules.dsets:185:__init__ Covid2dSegmentationDataset: validation mode, 20 uid's, 592 index slices, 157 lesions


NameError: name 'epochs' is not defined

We are overfitting quite severly by predicting almost everything to be a lesion. Lets try to add augmentation and see how it performs

## Augmentation Test

In [ ]:
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-flip', '--run-name=bb60,flip aug')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-offset', '--run-name=bb60,offset aug')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-scale', '--run-name=bb60,scale aug')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-rotate', '--run-name=bb60,rotate aug')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augment-noice', '--run-name=bb60,noise aug')
run('training.CovidSegmentationTrainingApp', f'--epochs={experiment_epochs}', f'--batch-size={batch_size}', '--augmented', '--run-name=bb60,all aug')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-11-14 13:48:59,115 INFO     pid:10583 nb:006:run Running: training.CovidSegmentationTrainingApp(['--num-workers=4', '--epochs=3', '--batch-size=32', '--augment-flip', '--run-name=bb60,flip aug']).main()
2020-11-14 13:48:59,127 INFO     pid:10583 training:174:init_model Using CUDA; 1 devices.
2020-11-14 13:48:59,159 INFO     pid:10583 modules.dsets:185:__init__ TrainingCovid2dSegmentationDataset: training mode, 177 uid's, 4379 index slices, 1080 lesions
2020-11-14 13:48:59,161 INFO     pid:10583 modules.dsets:226:__init__ TrainingCovid2dSegmentationDataset: (7, 60, 60) width_irc
2020-11-14 13:48:59,174 INFO     pid:10583 modules.dsets:185:__init__ Covid2dSegmentationDataset: validation mode, 20 uid's, 592 index slices, 157 lesions
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb: wandb version 0.10.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


2020-11-14 13:49:03,765 INFO     pid:10583 training:455:main Starting CovidSegmentationTrainingApp, Namespace(augment_flip=True, augment_noise=False, augment_offset=False, augment_rotate=False, augment_scale=False, augmented=False, batch_size=32, ct_window=None, epochs=3, num_workers=4, pad_type='zero', run_name='bb60,flip aug', val_cadence=1, width_irc=[7, 60, 60])


epoch,loss/trn,loss/val,metrics_val/miss_rate,metrics_val/fp_to_mask_ratio,pr_val/precision,pr_val/recall,pr_val/f1_score
1,0.4540,0.9667,0.0003,57.1078,0.0172,0.9997,0.0338
2,0.4501,0.9669,0.0009,57.4648,0.0171,0.9991,0.0336
3,0.4449,0.9662,0.0008,56.2887,0.0174,0.9992,0.0343


2020-11-14 13:49:11,616 INFO     pid:10583 training:297:log_metrics E1 CovidSegmentationTrainingApp
2020-11-14 13:49:11,617 INFO     pid:10583 training:326:log_metrics E1 trn      0.4540 loss, 0.4222 precision, 0.9903 recall, 0.5920 f1 score 0.0097 miss rate 1.3555 fp to label ratio
2020-11-14 13:49:26,615 INFO     pid:10583 training:297:log_metrics E1 CovidSegmentationTrainingApp
2020-11-14 13:49:26,617 INFO     pid:10583 training:335:log_metrics E1 val      0.9667 loss, 0.0172 precision, 0.9997 recall, 0.0338 f1 score 0.0003 miss rate 57.1078 fp to label ratio
2020-11-14 13:49:42,809 INFO     pid:10583 training:434:save_model Saved model params to saved-models/2020-11-14_13.48.59.10000.state
2020-11-14 13:49:42,811 INFO     pid:10583 training:439:save_model Saved model params to saved-models/2020-11-14_13.48.59.best.state
2020-11-14 13:49:42,816 INFO     pid:10583 training:444:save_model SHA1: dbe4b509dba9a86b316fdc851740c05ff285ca21
2020-11-14 13:49:50,273 INFO     pid:10583 trainin

# Bbox (7,100,100)

Fill cache for (7,100,100) bounding box

In [ ]:
# run('prepcache.CovidPrepCacheApp', '--width-irc', '7', '100', '100')